In [ ]:
import pandas as pd
import numpy as np
import pickle

import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.graph_objects as go

In [ ]:
with open('../data/nash_hosp_referrers_b.pickle', 'rb') as file:
    nash_hosp_referrers_b = pickle.load(file)

In [ ]:
nash_hosp_referrers_b['hospital'] = (
    nash_hosp_referrers_b['hospital']
    .str.replace('SETON CORPORATION', 'SAINT THOMAS')
    .str.replace('SAINT THOMAS WEST HOSPITAL', 'SAINT THOMAS')
    .str.replace('HCA HEALTH SERVICES OF TENNESSEE, INC.', 'HCA/TRISTAR')
    .str.replace('HTI MEMORIAL HOSPITAL CORPORATION', 'HCA/TRISTAR')
    .str.replace('NASHVILLE GENERAL HOSPITAL', 'NASHVILLE GENERAL')
    .str.replace('VANDERBILT UNIVERSITY MEDICAL CENTER', 'VUMC')
)

#add a column that calculates the total wait in days for each providers' referrals
nash_hosp_referrers_b['total_wait'] = (nash_hosp_referrers_b['transaction_count'] 
                                       * 
                                       nash_hosp_referrers_b['average_day_wait'])

In [ ]:
# Create an interactive table that allows selection of a specialty and list of providers in order of referrals going to other hospitals
@interact(Specialty = np.sort(nash_hosp_referrers_b
                              .loc[~nash_hosp_referrers_b['hospital'].str.contains('VUMC')]
                              .groupby(['from_npi',
                                        'first_name',
                                        'last_name',
                                        'credential',
                                        'location_address_state_name', 
                                        'location_address_city_name', 
                                        'specialty',
                                        'sub_specialty'])
                              .agg(total_providers = ('from_npi', 'count'),
                                   total_transactions = ('transaction_count', 'sum'),
                                   total_patients = ('patient_count', 'sum'))
                              .reset_index()
                              .sort_values('total_transactions', ascending = False)
                              ['specialty'].unique())
         )

def interactive_plot(Specialty):
    
    df = (nash_hosp_referrers_b
          .loc[(~nash_hosp_referrers_b['hospital'].str.contains('VUMC'))
               &
               (nash_hosp_referrers_b['specialty'] == Specialty)]
          .groupby(['from_npi',
                    'first_name',
                    'last_name',
                    'credential',
                    'location_address_state_name', 
                    'location_address_city_name', 
                    'specialty',
                    'sub_specialty'])
          .agg(total_providers = ('from_npi', 'count'),
               total_transactions = ('transaction_count', 'sum'),
               total_patients = ('patient_count', 'sum'),
               total_states = ('location_address_state_name', 'nunique'),
               total_wait = ('total_wait', 'sum')
               
          )
          .reset_index()
          .sort_values('total_transactions', ascending = False))
    
    df['avg_day_wait'] = round(df['total_wait'] / df['total_transactions'], 2)
    
    fig = go.Figure(data=[go.Table(
        header=dict(values= ['Provider NPI', 
                             'First Name',
                             'Last Name',
                             'Credential', 
                             'Sub-Specialty', 
                             'State', 
                             '# Non-VUMC Referral Transactions',
                             '# Patients',
                             'Average Wait (days)'],
                    fill_color='#2d5325',
                    font_color= '#ffffff',
                    align='left'),
        cells=dict(values=[df.from_npi,
                           df.first_name,
                           df.last_name,
                           df.credential,
                           df.sub_specialty, 
                           df.location_address_state_name,
                           df.total_transactions,
                           df.total_patients,
                           df.avg_day_wait],
                   fill_color='#e8f4e5',
                   align='left'))
    ])

    fig.show()